In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [33]:
#Load the dataset
data = pd.read_csv('Battery_dataset.csv')
data.drop_duplicates() 

,Time,Voltage,Current,Temperature,Capacity,WhAccu
0,1.9,4.17402,-1.20553,23.87099,0.00000,0.00000
1,2.0,4.16559,-1.08549,23.87099,-0.00003,-0.00013
2,2.1,4.16256,-1.05228,23.87099,-0.00006,-0.00026
3,2.2,4.16155,-1.04462,23.87099,-0.00009,-0.00038
4,2.3,4.16104,-1.04207,23.87099,-0.00012,-0.00050
...,...,...,...,...,...,...
85909,9.6,3.21554,0.00000,23.87099,-2.55821,-9.18608
85910,9.7,3.21554,0.00000,23.87099,-2.55821,-9.18608
85911,9.8,3.21554,0.00000,23.87099,-2.55821,-9.18608
85912,9.9,3.21554,0.00000,23.87099,-2.55821,-9.18608


In [81]:
# divide positive and negative values
p_values = df[df['Current'] >= 0]
n_values = df[df['Current'] < 0]

In [82]:
n_values

,Time,Voltage,Current,Temperature,Capacity,WhAccu
0,1.9,4.17402,-1.20553,23.87099,0.00000,0.00000
1,2.0,4.16559,-1.08549,23.87099,-0.00003,-0.00013
2,2.1,4.16256,-1.05228,23.87099,-0.00006,-0.00026
3,2.2,4.16155,-1.04462,23.87099,-0.00009,-0.00038
4,2.3,4.16104,-1.04207,23.87099,-0.00012,-0.00050
...,...,...,...,...,...,...
79909,2.8,2.79976,-9.94307,25.02773,-2.55711,-9.18301
79910,2.9,2.79993,-9.88433,25.02773,-2.55739,-9.18379
79911,3.0,2.79976,-9.83580,25.02773,-2.55767,-9.18456
79912,3.1,2.79976,-9.78216,25.02773,-2.55793,-9.18530


In [83]:
data1 = data.head(1200)
data1

,Time,Voltage,Current,Temperature,Capacity,WhAccu
0,1.9,4.17402,-1.20553,23.87099,0.00000,0.00000
1,2.0,4.16559,-1.08549,23.87099,-0.00003,-0.00013
2,2.1,4.16256,-1.05228,23.87099,-0.00006,-0.00026
3,2.2,4.16155,-1.04462,23.87099,-0.00009,-0.00038
4,2.3,4.16104,-1.04207,23.87099,-0.00012,-0.00050
...,...,...,...,...,...,...
1195,1.4,4.04238,-3.53486,23.87099,-0.03913,-0.15592
1196,1.5,4.04254,-3.53486,23.87099,-0.03922,-0.15632
1197,1.6,4.04271,-3.53486,23.87099,-0.03932,-0.15671
1198,1.7,4.04288,-3.53486,23.87099,-0.03942,-0.15711


In [84]:
# Preprocess the dataset by cleaning and filtering the data
data1 = data1.dropna()  # Remove rows with missing data
data1 = data1[data1['Current'] != 0]  # Remove rows with zero current

In [85]:
# Calculate the SOC using the Coulomb counting method
dt = 1
N = len(data1['Voltage'])
SOC = np.zeros(N)
SOC[0] = 1.0 # initial state of charge
Q = 3600 # battery capacity in Coulombs
IR = 0.5 
for i in range(1, N):
    dt = 1 # time difference assumed to be 1 second
    coulombs = (data['Current'][i] + data['Current'][i-1])/2*dt/Q # average current
    SOC[i] = SOC[i-1] - coulombs + IR*data['Current'][i]/Q 

In [86]:
# Print the estimated SOC
SOC
import pandas as pd
df_soc  = pd.DataFrame({'SOC':SOC})
print(df_soc)

           SOC
0     1.000000
1     1.000167
2     1.000318
3     1.000464
4     1.000609
...        ...
1195  1.194475
1196  1.194966
1197  1.195457
1198  1.195948
1199  1.196439

[1200 rows x 1 columns]


In [87]:
# Split the dataset into training and testing sets
X = data1[['Voltage', 'Current']]
y = df_soc
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [88]:
# Create a Decision Tree Regression model
model = DecisionTreeRegressor(max_depth=5)

# Train the model on the training set
model.fit(X_train, y_train)

# Evaluate the model on the testing set
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean squared error: {mse:.2f}")

# Evaluate the model on the testing set to measure its accuracy
score = model.score(X_test, y_test)
print(f"Model accuracy: {score:.2f}")

Mean squared error: 0.00
Model accuracy: 0.55


In [89]:
# Initialize variables
N = len(n_values['Voltage']) # number of data points
SOC = np.zeros(N) # state of charge
SOC[0] = 0.5 # initial state of charge
Q = 2200 # battery capacity in Coulombs
IR = 0.67 # internal resistance of the battery in Ohms

In [90]:
# Coulomb counting method
for i in range(1, N):
    dt = 1 # time difference assumed to be 1 second
    coulombs = (data['Current'][i]*1.67 + data['Current'][i-1])/2*dt/Q # average current
    SOC[i] = SOC[i-1] - coulombs + IR*data['Current'][i]/Q

In [91]:
SOC
import pandas as pd
df_soc  = pd.DataFrame({'SOC':SOC})
print(df_soc)

             SOC
0       0.500000
1       0.500355
2       0.500681
3       0.500999
4       0.501314
...          ...
62201  20.431523
62202  20.429710
62203  20.427897
62204  20.426084
62205  20.424272

[62206 rows x 1 columns]


In [101]:
sorted_list = sorted(SOC, reverse=True)
sorted_list

[20.546856753752486,
 20.546348317957033,
 20.545933282093397,
 20.545861725275213,
 20.545382283184306,
 20.54490535507067,
 20.544535131684306,
 20.54442900650249,
 20.54395265793431,
 20.543476501366126,
 20.543298372207026,
 20.543198116616114,
 20.543137563093396,
 20.543101376366117,
 20.543048189866116,
 20.543000926616124,
 20.542880088025207,
 20.542811091775206,
 20.542707539297933,
 20.54253363795702,
 20.54252535186612,
 20.542520807184296,
 20.542359157070656,
 20.542229943047932,
 20.54218467618429,
 20.54209326061612,
 20.542010195297927,
 20.54193888766157,
 20.54183571441156,
 20.541816499547938,
 20.541740186502487,
 20.541674259525198,
 20.541647832275206,
 20.541618991729756,
 20.54155955686611,
 20.541469403297924,
 20.541447389138845,
 20.54138756168429,
 20.541356195070662,
 20.541308042797926,
 20.54128274741157,
 20.541228715161562,
 20.541149967070652,
 20.54112003859339,
 20.54107121897974,
 20.541062624957025,
 20.54099247088883,
 20.54095791159339,
 20.5409

In [103]:
from sklearn.preprocessing import MinMaxScaler

# create a sample output array
output = sorted_list

# create a scaler object
scaler = MinMaxScaler(feature_range=(0, 100))

# fit and transform the data
scaled_output = scaler.fit_transform([[x] for x in output])

# convert the scaled output to a list
scaled_output = [x[0] for x in scaled_output]

# print the scaled output
print(scaled_output)


[100.0, 99.99746376301434, 99.99539343414067, 99.99503648632054, 99.99264487901375, 99.99026581221294, 99.9884190220108, 99.98788963636636, 99.98551346051977, 99.98313824242932, 99.98224967839512, 99.98174957210843, 99.98144751217224, 99.98126700144319, 99.9810016905236, 99.98076592663017, 99.98016314589302, 99.97981897098894, 99.97930241880051, 99.97843494444756, 99.97839361083102, 99.97837094053472, 99.97756457914038, 99.97692001912625, 99.9766942138332, 99.97623820434855, 99.97582384852602, 99.9754681437029, 99.97495348321884, 99.97485763346114, 99.97447696008982, 99.97414809567931, 99.97401626827954, 99.97387240260622, 99.97357592289183, 99.97312620865836, 99.97301639513822, 99.97271795705741, 99.97256149056489, 99.97232129194758, 99.97219511063813, 99.97192558085263, 99.9715327607119, 99.97138346809396, 99.97113994057118, 99.97109707089429, 99.97074712043046, 99.97057472784111, 99.97035430028974, 99.96988790128236, 99.96976598758826, 99.96959793959289, 99.96915458024361, 99.968958

In [104]:
import plotly.graph_objs as go
import plotly.offline as pyo
import pandas as pd

# Load the data
df = pd.read_csv('Battery_dataset.csv')
M = []
for i in range(1,len(n_values['Voltage'])):
    M = M + [i]
    
# Create a trace
trace = go.Scatter(y=scaled_output, x=M, mode='lines')

# Create a layout
layout = go.Layout(title='Random_Forest', xaxis=dict(title='Time(s)'), yaxis=dict(title='SOC'))

# Create a figure
fig = go.Figure(data=[trace], layout=layout)

# Save the plot to an HTML file
pyo.plot(fig, filename='my_plot.html')

'my_plot.html'

In [105]:
# Initialize variables
N = len(p_values['Voltage']) # number of data points
SOC = np.zeros(N) # state of charge
SOC[0] = 0.5 # initial state of charge
Q = 2200 # battery capacity in Coulombs
IR = 0.67 # internal resistance of the battery in Ohms

In [106]:
# Coulomb counting method
for i in range(1, N):
    dt = 1 # time difference assumed to be 1 second
    coulombs = (data['Current'][i]*1.67 + data['Current'][i-1])/2*dt/Q # average current
    SOC[i] = SOC[i-1] - coulombs + IR*data['Current'][i]/Q

In [107]:
SOC
import pandas as pd
df_soc  = pd.DataFrame({'SOC':SOC})
print(df_soc)

            SOC
0      0.500000
1      0.500355
2      0.500681
3      0.500999
4      0.501314
...         ...
23704  5.419238
23705  5.418887
23706  5.418536
23707  5.418186
23708  5.417835

[23709 rows x 1 columns]


In [109]:
from sklearn.preprocessing import MinMaxScaler

# create a sample output array
output = SOC

# create a scaler object
scaler = MinMaxScaler(feature_range=(0, 100))

# fit and transform the data
scaled_output = scaler.fit_transform([[x] for x in output])

# convert the scaled output to a list
scaled_output = [x[0] for x in scaled_output]

# print the scaled output
print(scaled_output)

[0.0, 0.007168325477998394, 0.013736138978995172, 0.02014012735319959, 0.02650514412547622, 0.03285847148001508, 0.039211798834553946, 0.04556125355375329, 0.05189897301436197, 0.05824056511030484, 0.06455138424103701, 0.07070634263493325, 0.07677162256637793, 0.08281352366234351, 0.0888554247583091, 0.09489345321894227, 0.10091974642097767, 0.10694991225834904, 0.11298794071898044, 0.11901423392101762, 0.12534962695829144, 0.13282259424848775, 0.1409546122860963, 0.14925431295834635, 0.157604689777747, 0.16596680185574364, 0.17432891393374028, 0.18269102601173515, 0.1910531380897318, 0.19941525016772843, 0.21135513270685102, 0.23438016022540786, 0.25823940189496675, 0.28241838535399033, 0.3066947925461374, 0.3310219065988047, 0.3553607100692151, 0.3797033861749579, 0.4040577975392967, 0.4284122089036355, 0.4546057287380538, 0.48656935512008914, 0.5192073756417415, 0.5521065991824496, 0.5850993738209453, 0.6181272625535179, 0.6511628814292774, 0.6842219249813724, 0.7172809685334673, 0.

In [110]:
import plotly.graph_objs as go
import plotly.offline as pyo
import pandas as pd

# Load the data
df = pd.read_csv('Battery_dataset.csv')
M = []
for i in range(1,len(p_values['Voltage'])):
    M = M + [i]
    
# Create a trace
trace = go.Scatter(y=scaled_output, x=M, mode='lines')

# Create a layout
layout = go.Layout(title='Random_Forest', xaxis=dict(title='Time(s)'), yaxis=dict(title='SOC'))

# Create a figure
fig = go.Figure(data=[trace], layout=layout)

# Save the plot to an HTML file
pyo.plot(fig, filename='my_plot.html')

'my_plot.html'